<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#从原始数据中提取所用信息" data-toc-modified-id="从原始数据中提取所用信息-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>从原始数据中提取所用信息</a></span></li><li><span><a href="#将已提取数据转化为推荐系统框架格式的数据：user-item-rating-timestamp" data-toc-modified-id="将已提取数据转化为推荐系统框架格式的数据：user-item-rating-timestamp-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>将已提取数据转化为推荐系统框架格式的数据：user item rating timestamp</a></span></li><li><span><a href="#保存歌单和歌曲信息" data-toc-modified-id="保存歌单和歌曲信息-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>保存歌单和歌曲信息</a></span></li><li><span><a href="#训练模型" data-toc-modified-id="训练模型-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>训练模型</a></span><ul class="toc-item"><li><span><a href="#使用协同过滤" data-toc-modified-id="使用协同过滤-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>使用协同过滤</a></span><ul class="toc-item"><li><span><a href="#建立映射字典" data-toc-modified-id="建立映射字典-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>建立映射字典</a></span></li><li><span><a href="#读取文件和训练" data-toc-modified-id="读取文件和训练-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>读取文件和训练</a></span></li><li><span><a href="#寻找最相近的10个user（此处是歌单）" data-toc-modified-id="寻找最相近的10个user（此处是歌单）-4.1.3"><span class="toc-item-num">4.1.3&nbsp;&nbsp;</span>寻找最相近的 10 个 user（此处是歌单）</a></span></li><li><span><a href="#针对user进行预测" data-toc-modified-id="针对user进行预测-4.1.4"><span class="toc-item-num">4.1.4&nbsp;&nbsp;</span>针对 user 进行预测</a></span></li></ul></li><li><span><a href="#使用矩阵分解" data-toc-modified-id="使用矩阵分解-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>使用矩阵分解</a></span></li></ul></li><li><span><a href="#模型存储和载入" data-toc-modified-id="模型存储和载入-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>模型存储和载入</a></span></li><li><span><a href="#不同的推荐算法评估" data-toc-modified-id="不同的推荐算法评估-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>不同的推荐算法评估</a></span></li><li><span><a href="#歌曲序列建模" data-toc-modified-id="歌曲序列建模-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>歌曲序列建模</a></span><ul class="toc-item"><li><span><a href="#训练得出歌曲的词向量" data-toc-modified-id="训练得出歌曲的词向量-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>训练得出歌曲的词向量</a></span></li><li><span><a href="#预测song2vec的相似度" data-toc-modified-id="预测song2vec的相似度-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>预测 song2vec 的相似度</a></span></li></ul></li></ul></div>

# 从原始数据中提取所用信息

In [ ]:
import json
import sys

def parse_song_line(music_file):
    data = json.loads(music_file)
    name = data['result']['name']
    tags = ','.join(data['result']['tags'])
    subscribed_count = data['result']['subcribedCount']
    if subscribed_count<100:
        return False
    playlist_id = data['result']['id']
    song_info = ''
    songs = data['result']['tracks']
    for song in songs:
        try:
            song_info += '\t'+':::'.join([str(song['id']), song['name'], song['artists'][0]['name'], str(song['popularity'])])
        except:
            continue
    return name+'##'+tags+'##'+str(playlist_id)+'##'+str(subscribed_count)+song_info

def parse_file(in_file, out_file):
    with open(out_file, 'w') as f:
        with open(in_file, 'r', encoding = 'utf-8') as g:
            for line in g.readlines():
                result = parse_song_line(line)
                if result:
                    print(result)
                    f.write(result.encode('utf-8').strip() + '\n')
                
parse_file('./playlist.detail.all/playlistdetail.all.json', './163_music_playlist.txt')

# 将已提取数据转化为推荐系统框架格式的数据：user item rating timestamp

In [ ]:
import json
import sys

def is_null(s):
    return len(s.split(','))>2

def parse_song_info(song_info):
    try:
        song_id, name, artist, popularity = song_info.split(':::')
        return ','.join([song_id, '1.0', '1300000'])
    ##评分时，因为原数据仅有是否收藏，因此分数统一写为1.0分
    except:
        return ''

def parse_playlist_line(in_line):
    try:
        contents = in_line.strip().split('\t')
        name, tags, playlist_id, subscribed_count = content[0].split('##')
        songs_info = map(lambda x: playlist_id+','+parse_song_info(x), contents[1:])
        ##这里是以一个歌单为标准格式中的user
        songs_info = filter(is_null, songs_info)
        return '\n'.join(songs_info)
    except:
        return False
    
def parse_file(in_file, out_file):
    with open(out_file, 'w') as out:
        with open(in_file, 'r', encoding = 'utf-8') as f:
            for line in f.readlines():
                result = parse_playlist_line(line)
                if result:
                    out.wirte(result.encode('utf-8').strip()+'\n')

parse_file("./163_music_playlist.txt", "./163_music_suprise_format.txt")

# 保存歌单和歌曲信息

In [ ]:
import pickle
import sys

def parse_playlist_get_info(in_line, playlist_dic, song_dic):
    contents = in_line.strip().split('\t')
    name, tags, playlist_id, subscribed_count = contents[0].split('##')
    playlist_dic[playlist_id] = name
    for song in contents[1]:
        try:
            song_id, song_name, artist, popularity = song.split(':::')
            song_dic[song_id] = song_name+'\t'+artist
        except:
            print('song format error')
            
def parse_file(in_file, out_playlist, out_song):
    playlist_dic = {}
    song_dic = {}
    with open(in_file, 'r') as f:
        for line in f.readlines():
            parse_playlist_get_info(line, playlist_dic, song_dic)
            with open(out_playlist, 'wb') as f:
                pickle.dump(playlist_dic, f)
            with open(out_song, 'wb') as f:
                pickle.dump(song_dic, f)

parse_file('./163_music_playlist.txt', 'playlist.pkl', 'song.pkl')

# 训练模型

## 使用协同过滤

In [1]:
from surprise import Dataset, Reader, KNNBaseline
import pickle
import os

### 建立映射字典

In [2]:
with open('popular_playlist.pkl', 'rb') as f:
    id_name_dic = pickle.load(f, encoding = 'utf-8')
name_id_dic = {values:key for key, values in id_name_dic.items()}

### 读取文件和训练

In [3]:
file_path = 'popular_music_suprise_format.txt'
reader = Reader(line_format = 'user item rating timestamp', sep = ',')
music_data = Dataset.load_from_file(file_path, reader = reader)
trainset = music_data.build_full_trainset()
#sim_options = {'name':'pearson_baseline', 'user_based':False}
algo = KNNBaseline()
algo.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [4]:
print(trainset.n_items, trainset.n_ratings, trainset.n_users)

50539 153250 1076


### 寻找最相近的10个user（此处是歌单）

In [5]:
##映射出内部id
current_playlist = list(name_id_dic.keys())[13]
playlist_id = name_id_dic[current_playlist]
playlist_inner_id = algo.trainset.to_inner_uid(playlist_id)

playlist_neighbors = algo.get_neighbors(playlist_inner_id, k=10)
print(playlist_neighbors)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 11]


In [6]:
print(' 与歌单%s 相似的10个歌单为\n'%(current_playlist))
for name in [id_name_dic[algo.trainset.to_raw_uid(neighbors)] for neighbors in playlist_neighbors]:
    print(name)

 与歌单老旧扬声器里的华语经典 相似的10个歌单为

当过千评论的华语翻唱遇上“原唱”【更新】
第四季中国好声音第三期原唱及翻唱
【华语】暖心物语 纯白思念
〖循环〗单曲循环是强迫症吗？
周杰伦地表最强演唱会2017520南京站曲目
简单的爱总是那么吸引人
别让时光偷走你上扬的嘴角
大神爱翻唱1：华语篇
国语VS闽南语 给你最强听觉冲击
雾霾诗集


### 针对user进行预测

In [7]:
##歌曲id映射
with open('popular_song.pkl', 'rb') as f:
    song_id_name_dic = pickle.load(f, encoding = 'utf-8')
song_name_id_dic = {values: keys for keys, values in song_id_name_dic.items()}

In [8]:
user_inner_id = 4
user_rating = trainset.ur[user_inner_id]
items = [item for (item, rating) in user_rating]
for song in items:
    print(algo.predict(user_inner_id, song, r_ui=1), song_id_name_dic[algo.trainset.to_raw_iid(song)])

user: 4          item: 478        r_ui = 1.00   est = 1.00   {'was_impossible': False} 听见下雨的声音	魏如昀
user: 4          item: 429        r_ui = 1.00   est = 1.00   {'was_impossible': False} 梦一场	萧敬腾
user: 4          item: 936        r_ui = 1.00   est = 1.00   {'was_impossible': False} 干杯	西瓜Kune
user: 4          item: 937        r_ui = 1.00   est = 1.00   {'was_impossible': False} 给自己的歌 (Live) - live	纵贯线
user: 4          item: 938        r_ui = 1.00   est = 1.00   {'was_impossible': False} 小半	陈粒
user: 4          item: 939        r_ui = 1.00   est = 1.00   {'was_impossible': False} 思念是一种病(Live) - live	张震岳
user: 4          item: 940        r_ui = 1.00   est = 1.00   {'was_impossible': False} 可以不可以	丁当
user: 4          item: 941        r_ui = 1.00   est = 1.00   {'was_impossible': False} 秋酿	房东的猫
user: 4          item: 616        r_ui = 1.00   est = 1.00   {'was_impossible': False} 退后	周杰伦
user: 4          item: 942        r_ui = 1.00   est = 1.00   {'was_impossible': False} 阴天	莫文蔚
user: 4        

## 使用矩阵分解

In [9]:
from surprise import NMF, Dataset, evaluate, Reader
import pickle

file_path = 'popular_music_suprise_format.txt'
reader = Reader(line_format = 'user item rating timestamp', sep = ',')
music_data = Dataset.load_from_file(file_path, reader = reader)
algo = NMF()
trainset = music_data.build_full_trainset()
algo.fit(trainset)

In [10]:
with open('popular_song.pkl', 'rb') as f:
    song_id_name_dic = pickle.load(f, encoding = 'utf-8')
song_name_id_dic = {value: key for key, value in song_id_name_dic.items()}

In [11]:
user_inner_id = 4
user_rating = trainset.ur[user_inner_id]
items = [item for (item, rating) in user_rating]
for song in items:
    print(algo.predict(user_inner_id, song, r_ui = 1.0), song_id_name_dic[trainset.to_raw_iid(song)])

user: 4          item: 478        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 听见下雨的声音	魏如昀
user: 4          item: 429        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 梦一场	萧敬腾
user: 4          item: 936        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 干杯	西瓜Kune
user: 4          item: 937        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 给自己的歌 (Live) - live	纵贯线
user: 4          item: 938        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 小半	陈粒
user: 4          item: 939        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 思念是一种病(Live) - live	张震岳
user: 4          item: 940        r_ui = 1.00   est = 1.00   {'reason': 'User and item are unkown.', 'was_impossible': True} 可以不可以	丁当
user: 4          item: 941      

# 模型存储和载入

In [12]:
import surprise
surprise.dump.dump('./recommedation.model', algo = algo)
algo = surprise.dump.load('./recommedation.model')

In [13]:
algo

(None,
 <surprise.prediction_algorithms.matrix_factorization.NMF at 0x23d30a74668>)

# 不同的推荐算法评估

In [14]:
import os
from surprise import Reader, Dataset
file_path = 'popular_music_suprise_format.txt'
reader = Reader(line_format = 'user item rating timestamp', sep = ',')
music_data = Dataset.load_from_file(file_path, reader = reader)
music_data.split(n_folds = 5)

In [15]:
music_data.raw_ratings[:10]

[('83315865', '1297650', 1.0, '1300000'),
 ('452880141', '326882', 1.0, '1300000'),
 ('43170093', '29812323', 1.0, '1300000'),
 ('435064329', '27927330', 1.0, '1300000'),
 ('26506598', '156228', 1.0, '1300000'),
 ('120213287', '185613', 1.0, '1300000'),
 ('137354902', '114037', 1.0, '1300000'),
 ('386638608', '27556170', 1.0, '1300000'),
 ('384483907', '288795', 1.0, '1300000'),
 ('4671456', '150623', 1.0, '1300000')]

In [16]:
from surprise import NormalPredictor
from surprise.model_selection import cross_validate

algo = NormalPredictor()
perf = cross_validate(algo, music_data, measures=['RMSE', 'MAE'])
perf

{'fit_time': (0.23438525199890137,
  0.27521848678588867,
  0.3204069137573242,
  0.29508042335510254,
  0.2812633514404297),
 'test_mae': array([0., 0., 0., 0., 0.]),
 'test_rmse': array([0., 0., 0., 0., 0.]),
 'test_time': (0.4140174388885498,
  0.5661826133728027,
  0.5198237895965576,
  0.3750178813934326,
  0.5368146896362305)}

In [17]:
from surprise import BaselineOnly
from surprise.model_selection import cross_validate

algo = BaselineOnly()
perf = cross_validate(algo, music_data, measures = ['RMSE', 'MAE'])

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


In [18]:
from surprise import KNNBasic
from surprise.model_selection import cross_validate

algo = KNNBasic()
perf = cross_validate(algo, music_data, measures = ['RMSE', 'MAE'])
perf

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'fit_time': (0.21914219856262207,
  0.17649316787719727,
  0.17188453674316406,
  0.18297123908996582,
  0.19187474250793457),
 'test_mae': array([0., 0., 0., 0., 0.]),
 'test_rmse': array([0., 0., 0., 0., 0.]),
 'test_time': (1.669342041015625,
  1.7331862449645996,
  1.6377153396606445,
  1.6377146244049072,
  1.7583675384521484)}

In [19]:
from surprise import KNNWithMeans
from surprise.model_selection import cross_validate

algo = KNNWithMeans()
perf = cross_validate(algo, music_data, measures = ['RMSE', 'MAE'])
perf

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'fit_time': (0.21146464347839355,
  0.23438572883605957,
  0.23605799674987793,
  0.23438024520874023,
  0.25174379348754883),
 'test_mae': array([0., 0., 0., 0., 0.]),
 'test_rmse': array([0., 0., 0., 0., 0.]),
 'test_time': (1.7267959117889404,
  1.602137565612793,
  1.6509218215942383,
  1.6209101676940918,
  1.76170015335083)}

In [20]:
from surprise import KNNBaseline
from surprise.model_selection import cross_validate

algo = KNNBaseline()
perf =cross_validate(algo, music_data, measures = ['RMSE', 'MAE'])
perf

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


{'fit_time': (0.8790373802185059,
  0.7429711818695068,
  0.8012757301330566,
  0.7202973365783691,
  0.7203996181488037),
 'test_mae': array([0., 0., 0., 0., 0.]),
 'test_rmse': array([0., 0., 0., 0., 0.]),
 'test_time': (2.33626651763916,
  2.923426389694214,
  2.19986629486084,
  2.1877710819244385,
  2.308274507522583)}

In [21]:
from surprise import SVD
from surprise.model_selection import cross_validate

algo = SVD()
perf = cross_validate(algo, music_data, measures = ['RMSE', 'MAE'])
perf

{'fit_time': (16.439724683761597,
  11.640626192092896,
  13.67088794708252,
  11.705570220947266,
  11.156784057617188),
 'test_mae': array([0.01682847, 0.01702269, 0.01687655, 0.01707745, 0.01677412]),
 'test_rmse': array([0.03656681, 0.03741667, 0.03712611, 0.03740246, 0.03685185]),
 'test_time': (0.5771915912628174,
  0.837554931640625,
  0.6204135417938232,
  0.5312743186950684,
  0.40626955032348633)}

In [22]:
from surprise import SVDpp
from surprise.model_selection import cross_validate

algo = SVDpp()
perf = cross_validate(algo, music_data, measures = ['RMSE', 'MAE'])
perf

{'fit_time': (749.1101882457733,
  754.2631189823151,
  748.7548055648804,
  750.3649053573608,
  747.4497148990631),
 'test_mae': array([0.0065434 , 0.00643753, 0.00659157, 0.0065203 , 0.00662982]),
 'test_rmse': array([0.01509312, 0.01475149, 0.0152187 , 0.0151625 , 0.01529261]),
 'test_time': (10.516133069992065,
  10.68801474571228,
  10.719265460968018,
  10.266117572784424,
  10.250494480133057)}

In [24]:
from surprise import NMF
from surprise.model_selection import cross_validate

algo = NMF()
perf = cross_validate(algo, music_data, measures = ['RMSE', 'MAE'])
perf

{'fit_time': (20.571425437927246,
  21.439573287963867,
  21.550687313079834,
  21.672035217285156,
  20.440409898757935),
 'test_mae': array([0., 0., 0., 0., 0.]),
 'test_rmse': array([0., 0., 0., 0., 0.]),
 'test_time': (0.3704102039337158,
  0.4072692394256592,
  0.36324357986450195,
  0.3692452907562256,
  0.556365966796875)}

# 歌曲序列建模

## 训练得出歌曲的词向量

In [1]:
import multiprocessing
import gensim
import sys
from random import shuffle

cores = multiprocessing.cpu_count()

def parse_playlist_get_sequence(in_line, playlist_sequence):
    song_sequence = []
    contents = in_line.strip().split('\t')
    for song in contents[1:]:
        ##列表的第0个元素是歌单的信息，不是歌曲的信息
        try:
            song_id, song_name, artists, popularity = song.split(':::')
            song_sequence.append(song_id)
        except:
            print('song format error')
    for i in range(len(song_sequence)):
        shuffle(song_sequence)
        playlist_sequence.append(song_sequence)
        
def train_song2vec(in_file, out_file):
    playlist_sequence = []
    with open(in_file, 'r', encoding = 'utf-8') as f:
        for line in f.readlines():
            parse_playlist_get_sequence(line, playlist_sequence)
    model = gensim.models.Word2Vec(sentences = playlist_sequence, size = 150, min_count = 3, window = 7, workers = cores)
    model.save(out_file)

train_song2vec('popular.playlist', 'song2vec.model')

song format error
song format error
song format error
song format error
song format error
song format error
song format error
song format error
song format error
song format error
song format error
song format error
song format error
song format error
song format error
song format error
song format error
song format error


## 预测song2vec的相似度

In [10]:
import pickle 
with open('popular_song.pkl', 'rb') as f:
    song_dic = pickle.load(f, encoding = 'utf-8')
    model = gensim.models.Word2Vec.load('song2vec.model')

In [18]:
song_id_list = list(song_dic.keys())[1000: 1500: 100]
for song_id in song_id_list:
    result_song_list = model.wv.most_similar(song_id)
    print(song_id, song_dic[song_id])
    print('\n相似歌曲和相似度分别为')
    for song in result_song_list:
        print('\t', song_dic[song[0]], song[1])
    print('\n')

394653 坐在巷口的那对男女	自然卷

相似歌曲和相似度分别为
	 ローリンガール	まらしぃ 0.5940958261489868
	 That Should Be Me	Justin Bieber 0.5920975208282471
	 Bones - Single	Alex G 0.5910210609436035
	 a little kiss	美图秀秀 0.5642493963241577
	 すすすす、すき、だあいすき	山根万理奈 0.551602840423584
	 Kate Moss	Maximilian Hecker 0.547816812992096
	 Right Here Where You Want It	X-Ray Dog 0.5320639610290527
	 末日幻象	Fayzz 0.53162682056427
	 L-O-V-E	Siena Wu 0.5258373618125916
	 CALL ME BABY	EXO 0.5220599174499512


413831050 星晴(Anky Hype Mix Re-drum)	Anky

相似歌曲和相似度分别为
	 水管的友情	黄雨勋 0.738114595413208
	 Try	派伟俊 0.7175805568695068
	 周杰伦歌曲串烧-文武贝改编版	文武贝 0.6703663468360901
	 明年今日 (Instrumental)	群星 0.6585835218429565
	 十二月的奇迹EXO 纯吉他	凡以三 0.6581536531448364
	 陶笛版本《再回首》	史峰SzeFung 0.6496715545654297
	  海阔天空 吉他纯音乐	徐涛 0.6465712785720825
	 红豆	群星 0.641079843044281
	 张杰《看月亮爬上来》钢琴版（流行爵士）	黎栩晴 0.6359755992889404
	 潇洒走一回	群星 0.6167184114456177


421110099 不再说分手 (Cover 王菀之)	刘瑞琦

相似歌曲和相似度分别为
	 哎呦	左颜 0.7056404948234558
	 我的烦恼是你 (Cover 徐若瑄)	刘瑞琦 0.5341656804084778
	 当我想你的时